# Deploy the model as an online endpoint

This Notebook deploy a ML trained by Notebook 01 as an realtime online endpoint. This enables applications to send a json request with data to get a prediction.

**Note**: This notebook is intended to be run by a Compute Instance in Azure Machine Learning

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION = "<subscription-id>"
RESOURCE_GROUP = "rg-we-atpws-aml"
WS_NAME = "aml-ws-atp001"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [24]:
import os
registered_model_name = "credit_default_prediction_standard" #Trained by Notebook 01

## Create a new online endpoint

In [25]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

In [26]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint credit-endpoint-78a2c255 provisioning state: Succeeded


In [27]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "credit-endpoint-78a2c255" with provisioning state "Succeeded" is retrieved


## Deploy the model to the endpoint

In [28]:
# Let's pick the latest version of the model
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)

In [29]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint credit-endpoint-78a2c255 exists


...............................................................................

## Test with a sample query

In [2]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

**Simulate a json payload that is to be send to the ML endpoint**

In [3]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8,0,0,0]
        ]
  }
}

Overwriting ./deploy/sample-request.json


## Test against ML endpoint

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)


## Test Realtime specfic Endpoint

In [4]:
#Excute step1 and -3, -2 of this cell

#hardcoding to endpoint
result = ml_client.online_endpoints.invoke(
    endpoint_name="credit-endpoint-78a2c255", #Hard-coded enpoint
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

print(result)  # This will show the raw response


[1, 0]


## Clean up resources - Delete endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)